# **0. 사전작업**
---

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

x_train의 각 샘플은 4개의 특성을 가지고 있으며, 총 8개의 샘플이 존재합니다. 

y_train은 각 샘플에 대한 레이블인데, 여기서는 0, 1, 2의 값을 가지는 것으로 보아 총 3개의 클래스가 존재합니다.

# **1. 소프트맥스 회귀 구현하기 (low-level)**
---

In [3]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


x_train의 크기는 8 x 4이며, y_train의 크기는 8 x 1입니다. 그런데 최종 사용할 레이블은 y_train에서 원-핫 인코딩을 한 결과여야 합니다. 클래스의 개수는 3개이므로 y_train에 원-핫 인코딩한 결과는 8 x 3의 개수를 가져야 합니다.

In [4]:
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
y_one_hot.shape

torch.Size([8, 3])

y_train에서 원-핫 인코딩한 것의 크기는 8 x 3입니다. 즉, W 행렬의 크기는 4 x 3이어야 합니다. W와 b를 선언하고, 옵티마이저로는 경사 하강법을 사용합니다. 그리고 학습률은 0.1로, 가설과 비용함수도 설정하여 총 1000번의 epoch를 수행합니다.

In [5]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum().mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 8.788898
Epoch  100/1000 Cost: 12.256920
Epoch  200/1000 Cost: 76.820190
Epoch  300/1000 Cost: 17.120235
Epoch  400/1000 Cost: 77.544296
Epoch  500/1000 Cost: 16.191706
Epoch  600/1000 Cost: 9.495394
Epoch  700/1000 Cost: 70.690643
Epoch  800/1000 Cost: 15.008781
Epoch  900/1000 Cost: 22.958021
Epoch 1000/1000 Cost: 2.516452


# **2. 소프트맥스 회귀 구현하기 (high-level)**
---

이번에 F.cross_entropy()를 사용하여 비용 함수를 구현해보겠습니다. F.cross_entropy()에 소프트맥스 함수가 포함되어 있기에 가설에서 소프트맥스 함수를 사용할 필요가 없습니다.

In [6]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


# **3. 소프트맥스 회귀 nn.Module로 구현하기**
---

이번에는 nn.Module로 소프트맥스 회귀를 구현해봅시다. 선형 회귀에서 구현에 사용했던 nn.Linear()를 사용합니다. output_dim이 1이었던 선형 회귀 때와 달리 output_dim은 이제 클래스의 개수여야 합니다.

In [7]:
# 모델 선언 및 초기화
# 4개의 특성을 가지고 3개의 클래스로 분류
# input_dim=4, output_dim=3
model = nn.Linear(4, 3)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573444
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473266
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289179
Epoch 1000/1000 Cost: 0.254148


# **4. 소프트맥스 회귀 클래스로 구현하기**
---

이제 소프트맥스 회귀를 nn.Module을 상속받은 클래스로 구현해봅시다.

In [8]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [9]:
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428280
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
